In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir("/Users/bjarnihaukurbjarnason/Documents/genki-signals")

Start recording video

In [ ]:
from genki_signals.signal_system import SignalSystem
from genki_signals.signal_sources import CameraSignalSource, MouseSignalSource, Sampler

source = Sampler(
    {'image': CameraSignalSource(resolution=(720, 480))},
    sample_rate=100
)
system = SignalSystem(source, [], update_rate=60)
system.start()

Display the video

In [ ]:
from genki_signals.signal_frontends import WidgetFrontend, Video

video = Video("image")

frontend = WidgetFrontend(system, widgets=[video])

video.widget

In [ ]:
source2 = Sampler({'mouse': MouseSignalSource()}, sample_rate=100)
system2 = SignalSystem(source2, [])
system2.start()

In [ ]:
from genki_signals.signal_frontends.visualization import Line, Scatter, Bar, Histogram

l = Line("timestamp", "mouse")
s = Scatter(("mouse", 0), ("mouse", 1), x_range=(0,2560), y_range=(0,1440))
b = Bar("mouse", y_range=(0,2560), x_names=["x", "y"])
h = Histogram(("mouse", 0), lookback_size=1000)

frontend2 = WidgetFrontend(system2, widgets=[l, s, b, h])

In [ ]:
l.widget

In [ ]:
s.widget

In [ ]:
b.widget

In [ ]:
h.widget

Read audio data from the mic and compute the fourier transform

In [ ]:
from genki_signals.signal_sources import MicSignalSource
import genki_signals.signal_functions as sf

mic_source = MicSignalSource()
derived = [
    sf.FourierTransform('audio', name='spectro', window_size=2048, window_overlap=1024)
]

system3 = SignalSystem(
    mic_source,
    derived
)
system3.start()

Plot the spectrogram

In [ ]:
#!pip install genki_widgets

In [ ]:
import genki_widgets as gw

spectro = gw.Spectrogram(
    ("spectro", 0),
    window_size = 2048,
    sample_rate = mic_source.sample_rate,
    n_visible_windows = 500,
    colormap_max = 3,
)

frontend3 = WidgetFrontend(system3)
# should be able to do widgets=[spectro] but there is some strange bug which requires changing
# the databuffer to a dictionary first, something about repr of databuffer failing because the internal
# values are lists and therefore have no property shape
frontend3.register_update_callback(id(spectro), lambda d: spectro.update(d.as_dict()))

In [ ]:
spectro